In [1]:
class Data:
    def __init__(self, *args):
        if (len(args) == 3):
            self.radius = 1
            self.placeHolders = []
            self.Y = -1        
            self.space = args[0]
            self.initSensorNum = args[1]
            self.epsilon = args[2]
            self.SensorPlaceHolderSetup()
            self.InitializeData()

        if (len(args) == 5):
            self.radius = 1
            self.placeHolders = []
            self.grid = args[0]
            self.Y = args[1]
            self.space = args[2]
            self.initSensorNum = args[3]
            self.epsilon = args[4]
            self.SensorPlaceHolderSetup()
            
    def frange(self, start, stop, step):
        steps = []
        while start <= stop:
            steps.append(start)
            start +=step
            
        return steps

    def SensorPlaceHolderSetup(self):
        Xs = self.frange(0, self.space[0], self.epsilon)
        Ys = self.frange(0, self.space[1], self.epsilon)
            
        for x in Xs:
          for y in Ys:
            self.placeHolders.append([x, y])
        

    def InitializeData(self): #SensorConfigurationSetup(self):
        Xs = self.frange(0, self.space[0], self.epsilon)
        Ys = self.frange(0, self.space[1], self.epsilon)
        self.grid = np.zeros(len(Xs) * len(Ys)).tolist()

        i = 0
        while i < self.initSensorNum:
            cell = random.randrange(len(self.grid))
            if self.grid[cell] == 0:
                self.grid[cell] = 1
                i += 1
        
        # self.grid = np.zeros(len(self.placeHolders)).tolist()

        # i = 0
        # while i < self.initSensorNum:
        #     cell = random.randrange(len(self.grid))
        #     if self.grid[cell] == 0:
        #         self.grid[cell] = 1
        #         i += 1

    def makeBoundaries(self, epsilon, minValue, maxValue):
        m = epsilon * np.ceil(float(1/epsilon) * minValue)
        M = epsilon * np.floor(float(1/epsilon) * maxValue)
        return m, M

    def GetSensorConfiguration(self):
        from collections import Counter
        sensorLocations, sensorTypes = self.GetSensorLocations()

        summaryDict = Counter(sensorTypes)

        # TODO: DIFFERENT SENSOR TYPE DEFINITIONS SHOULD BE ADDED HERE:
        configurationSummary = []
        for key in summaryDict:
            if (key == 1):
                configurationSummary.append(['motion sensors', summaryDict[key]])

            elif (key == 2):
                configurationSummary.append(['beacon sensors', summaryDict[key]])

        configurationDetails = []
        for index, loc in enumerate(sensorLocations):
            if (sensorTypes[index] == 1):
                configurationDetails.append(tuple([loc, 'kitchen', 'motion sensors']))

            elif (sensorTypes[index] == 2):
                configurationDetails.append(tuple([loc, 'kitchen', 'beacon sensors']))

        return [[configurationSummary, [tuple(configurationDetails)]], self.radius]


    def GetSensorLocations(self):
        sensorLocations = []
        sensorTypes = []
        for index, sensorIndicator in enumerate(self.grid):
            if (sensorIndicator > 0):
                sensorLocations.append(self.placeHolders[index])
                sensorTypes.append(sensorIndicator)

        return sensorLocations, sensorTypes
    

class BayesianOptimization:
    D = []

    def __init__(self, Data_path, epsilon, initSensorNum, maxSensorNum, radius, sampleSize):
        self.epsilon = epsilon
        self.Data_path = Data_path
        self.initSensorNum = initSensorNum
        self.maxSensorNum = maxSensorNum
        self.radius = radius
        self.sensor_distribution, self.types, self.space, self.rooms, self.agentTraces = self.ModelsInitializations()
      
        for i in range(sampleSize):
            data = Data(self.space, self.initSensorNum, self.epsilon)
            data.Y = self.Objective(data, True, False, False, 1)
            self.D.append(data)

        from sklearn.gaussian_process import GaussianProcessRegressor
        self.model = GaussianProcessRegressor()
        self.GetModelPosterior()

    def GetXY(self):
        X = []
        y = []
        for data in self.D:
            X.append(data.grid)
            y.append(data.Y)

        return X, y

    def GetModelPosterior(self):
        X, y = self.GetXY()
        self.model.fit(X, y)

    def ModelSampling(self, input_X):
        # catch any warning generated when making a prediction
        X = []
        y = []
        for data in input_X:
            if (type(data) is list):
                data = data[0]

            X.append(data.grid)
            y.append(data.Y)   

        return self.model.predict(X, return_std=True)

    # probability of improvement acquisition function
    def Acquisition(self, Xsamples):
        yhat, _ = self.ModelSampling(self.D)
        best = max(yhat)
        # calculate mean and stdev via surrogate function
        mu, std = self.ModelSampling(Xsamples)
        # print(mu, std)
        # mu = mu[:, 0]
        # calculate the probability of improvement
        probs = norm.cdf((mu - best) / (std+1E-9))
        # print(probs)
        return probs

    # optimize the acquisition function
    def OptimizeAcquisition(self):

        Xsamples = []
        for i in range(sampleSize):
            Xsamples.append(Data(self.space, self.initSensorNum, self.epsilon))

        # Xsamples = np.asarray(Xsamples)
        # Xsamples = Xsamples.reshape(len(Xsamples), 1)
        # print(Xsamples)
        scores = self.Acquisition(Xsamples)
        ix = argmax(scores)
        
        return self.D[ix]


    def ModelsInitializations(self):
        #----- Space and agent models -----: 
        simworldname = self.Data_path + '/Configuration Files/simulationWorld2.xml'
        agentTraces = []
        directory = os.fsencode(self.Data_path + 'Agent Trace Files Test/')
            
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".csv"): 
                agentTraces.append(self.Data_path + 'Agent Trace Files Test/' + filename)

        # Parsing the space model: 
        space, rooms = pf.ParseWorld(simworldname)

        xs = []
        for i in space:
          for j in i:
            xs.append(j)
        A = list(set(xs))
        A.sort()
        space = [A[-1], A[-2]]

        # User parameters 
        types, sensor_distribution = pf.GetUsersParameters()

        roomsList = []
        for room in sensor_distribution:
            roomsList.append(room)
              
        return sensor_distribution, types, space, rooms, agentTraces


    # TODO
    def X_next(self, ):
        sample = Data()



        return sample

    def Objective(self, sample, simulateMotionSensors, simulateEstimotes, Plotting, iteration):       
        files = []

        if (runningOnGoogleColab == True):
            sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
            Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
            
        else:
            sys.path.append('../../Codes/Sensor Simulator/')
            Data_path = '../../Codes/Sensor Simulator/'

        all_sensors = set([])


        for agentTrace in self.agentTraces:
            filecoding = ' '#"_" + str(iteration) + "_c" + str(index + 1) + '(' + self.mode + ')'
            df_ = sim_sis.RunSimulator(self.space, self.rooms, agentTrace, sample.GetSensorConfiguration() , simulateMotionSensors, simulateEstimotes, Plotting, filecoding , self.Data_path)
            dataFile, sensors = self.PreProcessor(df_)
            all_sensors.update(sensors)
            files.append(dataFile)
            
        
        if (runningOnGoogleColab == True):
            sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')

        else:
            sys.path.append('../../Codes/CASAS/AL-Smarthome')

        import al
        import imp
        imp.reload(al)
        all_sensors = list(all_sensors)

        accuracy = (al.leave_one_out(files, all_sensors)[0]) * 100

        if accuracy < 0:
            accuracy = 0

        return accuracy

    def frange(self, start, stop, step):
        steps = []
        while start <= stop:
            steps.append(start)
            start +=step
            
        return steps

    def MakeSensorCombinations(self, start, end, epsilon, sensorType, room):
        a1, b1 = self.makeBoundaries(epsilon, start[0], end[0])
        a2, b2 = self.makeBoundaries(epsilon, start[1], end[1])    
        Xs = self.frange(a1, b1, epsilon)
        Ys = self.frange(a2, b2, epsilon)
        
        points = list(itertools.product(list(itertools.product(Xs, Ys)), [room], [sensorType[0]])) 
        C = itertools.combinations(points, distribution[room][types.index(sensorType)])

        return C

    def PreProcessor(self, df):
        # df['motion sensors'] = df['motion sensors'].apply(ast.literal_eval)
        df['motion sensors'] = df['motion sensors'].apply(lambda s: list(map(int, s)))
        # df['beacon sensors'] = df['beacon sensors'].apply(ast.literal_eval)
        try:
          df['beacon sensors'] = df['beacon sensors'].apply(lambda s: list(map(int, s)))
        except:
          pass

        sensors = set([])

        previous_M = None
        previous_B = None
        output_file = []

        for index, row in df.iterrows():
          T = row['time']
          M = row['motion sensors']
          try:
            B = row['beacon sensors']
          except:
            pass

          Activity = row['activity']
          Activity = Activity.replace(' ', '_')
          MotionSensor_Names = []
          sensorNames = []
          MotionSensor_Message = []
          BeaconSensor_Names = []
          BeaconSensor_Message = []
          

          # time = convertTime(T)
          time = "2020-06-16 " + T + ".00"

          # Motion Sensor
          for i in range(len(M)):
            sensorNames.append(self.Name(i, 'M'))
            if M[i] == 1:
              if (previous_M != None):
                if (previous_M[i] == 0):
                  MotionSensor_Names.append(self.Name(i,'M'))
                  MotionSensor_Message.append('ON')

              else:
                MotionSensor_Names.append(self.Name(i,'M'))
                MotionSensor_Message.append('ON')

            if previous_M != None:
              if M[i] == 0 and previous_M[i] == 1:
                MotionSensor_Names.append(self.Name(i,'M'))
                MotionSensor_Message.append('OFF')

          previous_M = M
          # Beacon Sensor

          try:
            for i in range(len(B)):
              sensorNames.append(self.Name(i, 'B'))
              if B[i] == 1:
                BeaconSensor_Names.append(self.Name(i,'B'))
                BeaconSensor_Message.append('ON')
              if previous_B != None:
                if B[i] == 0 and previous_B[i] == 1: 
                  BeaconSensor_Names.append(self.Name(i,'B'))
                  BeaconSensor_Message.append('OFF')
            previous_B = B

          except:
            pass

          for m in range(len(MotionSensor_Names)):
            output_file.append(time +' '+ MotionSensor_Names[m] + ' ' + MotionSensor_Names[m] + ' ' + MotionSensor_Message[m] + ' ' + Activity)
            
          for s in sensorNames:
              sensors.add(s)

        return output_file, list(sensors)

    #returns the name of the sensor
    def Name(self, number, typeSensor):
        if number < 10:
          return typeSensor + str(0) + str(number)
        else:
          return typeSensor + str(number)

    #converts epoch time to human readable
    def convertTime(self, posix_timestamp):
        tz = pytz.timezone('MST')
        dt = datetime.fromtimestamp(posix_timestamp, tz)
        time = dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        return time

In [ ]:
global runningOnGoogleColab
runningOnGoogleColab = True

if __name__ ==  '__main__':
    import sys
    
    if (runningOnGoogleColab == True):
        from google.colab import drive    
        drive.mount('/content/gdrive', force_remount=True)
        Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
        
    else:
        Data_path = '../../Codes/Sensor Simulator/'
        sys.path.append('../../Codes/Sensor Simulator/')
        
    from ipywidgets import IntProgress
    from IPython.display import display
    from scipy.stats import norm
    from numpy import argmax
    import SIM_SIS_Libraries.SensorsClass
    import SIM_SIS_Libraries.SIM_SIS_Simulator as sim_sis
    import SIM_SIS_Libraries.ParseFunctions as pf
    import itertools
    import numpy as np
    import pandas as pd
    import sys
    import SIM_SIS_Libraries.PreDeploymentEvaluation as pde
    import copy
    from datetime import datetime
    import pytz
    import ast
    import os
    import random

    iteration = 1000
    # random_seed = 0.1234
    # random.seed(random_seed)

    # The distance between two nodes in the space grid:
    epsilon = 1
    
    # initial and max sensor numbers
    initSensorNum = 10
    maxSensorNum = 18

    # radius of the motion sensors
    radius = 1

    finalResults = []
    
    sampleSize = 100

    f = IntProgress(min=0, max=iteration) # instantiate the bar
    display(f) # display the bar

    
    BO =  BayesianOptimization(Data_path, 
                               epsilon, 
                               initSensorNum, 
                               maxSensorNum, 
                               radius,
                               sampleSize)


    for epoch in range(iteration): 
        f.value += 1

        x = BO.OptimizeAcquisition()
        # sample the point
        actual = BO.Objective(x, True, False, False, 1)

        # summarize the finding for our own reporting
        est, _ = BO.ModelSampling([x])

        print('>x=%.3f, f()=%3f, actual=%.3f' % (x.Y, est, actual))
        # add the data to the dataset
        # BO.D.append(Data(x.grid, x.Y, BO.D[0].space, BO.D[0].initSensorNum, BO.D[0].epsilon))
        BO.D.append(x)
        
        # update the model
        BO.GetModelPosterior()

        

Mounted at /content/gdrive


IntProgress(value=0, max=1000)

>x=43.843, f()=43.843103, actual=47.388
>x=43.843, f()=43.843103, actual=38.702
>x=43.843, f()=43.843103, actual=36.596
>x=43.843, f()=43.843103, actual=41.381
>x=43.843, f()=43.843103, actual=46.552
>x=43.843, f()=43.843103, actual=35.713
>x=43.843, f()=43.843103, actual=40.890
>x=43.843, f()=43.843103, actual=35.352
>x=43.843, f()=43.843103, actual=42.641
>x=43.843, f()=43.843103, actual=44.600
>x=43.843, f()=43.843103, actual=36.236
>x=43.843, f()=43.843103, actual=35.263
>x=43.843, f()=43.843103, actual=41.602
>x=43.843, f()=43.843103, actual=41.900
>x=43.843, f()=43.843103, actual=39.438
>x=43.843, f()=43.843103, actual=27.098
>x=43.843, f()=43.843103, actual=36.128
>x=43.843, f()=43.843103, actual=38.441
>x=43.843, f()=43.843103, actual=44.500
>x=43.843, f()=43.843103, actual=37.884
>x=43.843, f()=43.843103, actual=39.367
>x=43.843, f()=43.843103, actual=26.491
>x=43.843, f()=43.843103, actual=45.113
>x=43.843, f()=43.843103, actual=39.253
>x=43.843, f()=43.843103, actual=36.294
